In [1]:
import pandas as pd
import gensim.models.keyedvectors as word2vec
from numpy.linalg import norm
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA

In [2]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [3]:
df = pd.read_csv("./word2vec.csv",header=0, keep_default_na=False)
#df = pd.read_csv("./word2vec_debiased.csv",header=0, keep_default_na=False)
model =  word2vec.KeyedVectors.load_word2vec_format('./word2vec_50k.bin', binary=True)

In [4]:
df.head()

,word
0,prison
1,agony
2,vomit
3,rotten
4,cancer


In [5]:
# calculate bias direction when we have group of words not pairs
def groupBiasDirection(gp1, gp2):
    print(gp1,gp2)
    dim = len(model["he"])
    g1,g2 = np.zeros((dim,), dtype=float), np.zeros((dim,), dtype=float)
    for p in gp1:
        p = p.strip()
        if p not in model:
            continue
        p_vec = model[p]/norm(model[p])
        g1 = np.add(g1,p_vec)

    for q in gp2:
        q = q.strip()
        if q not in model:
            continue
        q_vec = model[q]/norm(model[q])
        g2 = np.add(g2,q_vec) 

    g1, g2 = g1/norm(g1), g2/norm(g2)
    return (g1,g2)

In [18]:
gender_bias = [("he","him","boy"),("she","her","girl")]
eco_bias = [("rich","wealthy"),("poor","impoverished")]
race_bias = [("african","black"),("european","white")]

In [19]:
g1,g2 = groupBiasDirection(gender_bias[0], gender_bias[1])
g3,g4 = groupBiasDirection(eco_bias[0], eco_bias[1])
g5,g6 = groupBiasDirection(race_bias[0], race_bias[1])

(('he', 'him', 'boy'), ('she', 'her', 'girl'))
(('rich', 'wealthy'), ('poor', 'impoverished'))
(('african', 'black'), ('european', 'white'))


In [20]:
df["semantic"] = None # encode semantic meaning by projecting to principal component
df["gender"] = None
df["eco"] = None
df["race"] = None

In [21]:
# calculaue principal component
matrix = []
for w in df["word"].tolist():
    matrix.append(model[w])
matrix = np.array(matrix)
pca = PCA(n_components=2)
pca.fit(matrix)
g = pca.components_[0]

In [22]:
words = df["word"].tolist()
for index, row in df.iterrows():
    w = row["word"]
    df.set_value(index, "gender", round(cosine(g1,model[w])-cosine(g2,model[w]),4))
    df.set_value(index, "eco", round(cosine(g3,model[w])-cosine(g4,model[w]),4))
    df.set_value(index, "race", round(cosine(g5,model[w])-cosine(g6,model[w]),4))
    df.set_value(index, "semantic", round(cos_sim(g,model[w]),5)) 

In [23]:
df.head()

,word,semantic,gender,eco,race
0,prison,0.16641,-0.0591,0.1214,0.0044
1,agony,-0.05575,-0.0026,0.0982,-0.0137
2,vomit,0.12755,0.0063,0.0706,0.0512
3,rotten,-0.1153,-0.0595,0.1722,0.0205
4,cancer,0.21116,0.0437,-0.0243,-0.0257


In [26]:
df = df.sort_values('semantic')
df_final = df[['word','gender','eco','race']]

In [28]:
gen_thresh=0.1
eco_thresh=0.1
race_thresh=0.05

In [34]:
df_final = df_final[(abs(df["gender"]) > gen_thresh) & (abs(df["eco"]) > eco_thresh) & (abs(df["race"]) > race_thresh)]

In [35]:
df_final.to_csv("../data/mutliple_biases.csv", encoding='utf-8', index=False)